## 데이터 솎아내기

In [ ]:
import json 
import os
import pandas as pd
from pycocotools.coco import COCO

In [ ]:
coco = COCO('../../train.json')

train_df = pd.DataFrame()

image_ids = []
class_name = []
class_id = []
x_min = []
y_min = []
x_max = []
y_max = []
classes = ["General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]
for image_id in coco.getImgIds():
        
    image_info = coco.loadImgs(image_id)[0]
    ann_ids = coco.getAnnIds(imgIds=image_info['id'])
    anns = coco.loadAnns(ann_ids)
        
    file_name = image_info['file_name']
        
    for ann in anns:
        image_ids.append(file_name)
        class_name.append(classes[ann['category_id']])
        class_id.append(ann['category_id'])
        x_min.append(float(ann['bbox'][0]))
        y_min.append(float(ann['bbox'][1]))
        x_max.append(float(ann['bbox'][0]) + float(ann['bbox'][2]))
        y_max.append(float(ann['bbox'][1]) + float(ann['bbox'][3]))

train_df['image_id'] = image_ids
train_df['class_name'] = class_name
train_df['class_id'] = class_id
train_df['x_min'] = x_min
train_df['y_min'] = y_min
train_df['x_max'] = x_max
train_df['y_max'] = y_max



In [ ]:
train_df

In [ ]:
z_score_limit = 3.5 # sigma 범위

index_ = pd.DataFrame()
mean_value = train_df['image_id'].value_counts().mean()
std_value = train_df['image_id'].value_counts().std()

index_['z_score'] = (train_df['image_id'].value_counts()-mean_value)/std_value
removed_index = [i for i in index_.index if index_['z_score'][i] > 3.5]


In [ ]:
removed_index

In [ ]:
len(removed_index)

## changed_train.json 생성

In [ ]:
# 새 파일 생성
root = '../../dataset/'
file_name = 'train.json'
save_name = 'changed_train.json'
train_df = pd.DataFrame

with open(os.path.join(root,file_name),'r') as f:
    coco_json = json.load(f)


In [ ]:
coco_json.keys()

In [ ]:
coco_json['images'][189:192]  # 원본 데이터 images key

In [ ]:
coco_info = coco_json['info']
coco_licenses = coco_json['licenses']
coco_images = [ i for i in coco_json['images'] if i['file_name'] not in removed_index]
coco_categories = coco_json['categories']
coco_annotations = coco_json['annotations']


In [ ]:
coco_images[185:188] # 0190.jpg 데이터 누락

In [ ]:
new_data = {
    'info': coco_info,
    'licenses': coco_licenses,
    'images': coco_images,
    'categories': coco_categories,
    'annotations': coco_annotations
}

In [ ]:
new_data.keys()

In [ ]:
# 새 파일 생성
root = 'C:\\Users\\leeyu\\Desktop\\'
file_name = 'train.json'
save_name = 'changed_train.json'
train_df = pd.DataFrame
output_path = os.path.join(root,save_name)

with open(output_path,'w') as f:
    json.dump(new_data, f, indent=2)